## Population Weighted Aggregation of Temperature and Precipitation data 
* guide: https://climateestimate.net/content/example-step3.html
* author: Abbie Boatwright
* date last updated: October 28th, 2022 

### To set up your virtual enviro run the following commands in terminal: 
* conda create -n grid_env
* conda active grid_env
* conda install -c conda-forge esmpy
* pip install xesmf
* pip install jupyter
* conda install -c conda-forge xagg
* conda install -c conda-forge regionmask cartopy pygeos
* conda install -c conda-forge rioxarray 
* ipython kernel install --name grid_env --user

In [ ]:
import xagg as xa
import xarray as xr 
import numpy as np
import regionmask
import geopandas as gpd
import pandas as pd
import os
import rioxarray
import time 
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir('/Users/Abbie/Dropbox/tree_lab_collab/data/raw')

In [3]:
#load country shapefile 
country_boundaries = gpd.read_file('world_shapefile/World_Countries__Generalized_.shp')

In [4]:
#read precip and temp data 
precip_g = xr.open_dataset('noaa_weather_data/precip.mon.total.v501.nc')
precip_g = precip_g.assign_coords(lon=(((precip_g.lon + 180) % 360) - 180)).sortby('lon')

temp_g = xr.open_dataset('noaa_weather_data/air.mon.mean.v501.nc')
temp_g = temp_g.assign_coords(lon=(((temp_g.lon + 180) % 360) - 180)).sortby('lon')

In [5]:
precip_g

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 1416)
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * time     (time) datetime64[ns] 1900-01-01 1900-02-01 ... 2017-12-01
Data variables:
    precip   (time, lat, lon) float32 ...
Attributes:
    Conventions:    CF-1.0
    title:          Terrestrial Air Temperature and Precipitation: V4.01
    version:        5.01
    dataset_title:  Terrestrial Air Temperature and Precipitation: 1900-2017 ...
    history:        created 12/2018 by CAS NOAA/ESRL PSD
    Source:         http://climate.geog.udel.edu/~climate/html_pages/download...
    References:     https://www.psl.noaa.gov/data/gridded/data.UDel_AirT_Prec...

In [ ]:
#load population data 
pop_g = xr.open_dataset('gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11_totpop_1_deg_nc/gpw_v4_population_density_adjusted_rev11_1_deg.nc')
pop_g = pop_g.sel(raster = 4) #selecting the year 2015 
pop_g = pop_g.rename({'longitude': 'lon','latitude': 'lat','UN WPP-Adjusted Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 1 degree': 'pop'})
pop_g = pop_g.drop('raster')

In [ ]:
# load agricultural data 
agr_g = xr.open_dataset('gl-croplands-geotif/cropland.nc')

In [ ]:
# creating population weightmap of pixels to polygons for precip 
# takes about 6 minutes to run 
pop_weightmap_precip =xa.pixel_overlaps(precip_g,country_boundaries, weights=pop_g.pop,subset_bbox=False)

In [ ]:
# creating agricultural (crop) weightmap of pixels to polygons for precip 
# takes about 6 minutes to run 
agr_weightmap_precip =xa.pixel_overlaps(precip_g,country_boundaries, weights=agr_g.Band1,subset_bbox=False)

In [ ]:
os.chdir('/Users/Abbie/Dropbox/tree_lab_collab/data/clean/climate')

In [ ]:
# aggregate precip using pop weightmap 
pop_aggregated_precip = xa.aggregate(precip_g, pop_weightmap_precip)
# aggregate precip using agr weightmap 
agr_aggregated_precip = xa.aggregate(precip_g, agr_weightmap_precip)


#save to CSV
pop_aggregated_precip.to_csv('precip_pop_agg.csv')
agr_aggregated_precip.to_csv('precip_agr_agg.csv')

In [ ]:
# create weightmap of pixels to polygons for temp 
pop_weightmap_temp =xa.pixel_overlaps(temp_g,country_boundaries, weights=pop_g.pop,subset_bbox=False)

In [ ]:
# create weightmap of pixels to polygons for temp 
agr_weightmap_temp =xa.pixel_overlaps(temp_g,country_boundaries, weights=agr_g.Band1,subset_bbox=False)

In [ ]:
# aggregate temp using pop weightmap 
pop_aggregated_temp = xa.aggregate(temp_g, pop_weightmap_temp)
# aggregate temp using agr weightmap 
agr_aggregated_temp = xa.aggregate(temp_g, agr_weightmap_temp)

#save to csv 
pop_aggregated_temp.to_csv('temp_pop_agg.csv')
agr_aggregated_temp.to_csv('temp_agr_agg.csv')

In [ ]:
# create weightmap of pixels to polygons for temp  without weights
unweightmap_temp = xa.pixel_overlaps(temp_g,country_boundaries,subset_bbox=False)

In [ ]:
# aggregate temp using weightmap 
unweighted_aggregated_temp = xa.aggregate(temp_g, unweightmap_temp)

#save to csv 
unweighted_aggregated_temp.to_csv('unweighted_temp_agg.csv')#_northernhemi.csv')

In [ ]:
# create weightmap of pixels to polygons for precip  without weights
unweightmap_precip =xa.pixel_overlaps(precip_g,country_boundaries,subset_bbox=False)

In [ ]:
# aggregate precip using weightmap 
unweighted_aggregated_precip = xa.aggregate(precip_g, unweightmap_precip)

#save to csv 
unweighted_aggregated_precip.to_csv('unweighted_precip_agg.csv')#_northernhemi.csv')

In [1]:
st = time.time()

NameError: name 'time' is not defined

In [ ]:
# how to time code: 

# st = time.time()

# *insert code* 
# et = time.time()
# elapsed_time = et - st
# print('Execution time:', elapsed_time, 'seconds')